To Address RQ4, we ran some statistical tests in order to find correlation and impact of variables. First, we read data and remove noises

In [1]:
import numpy as np
import pandas as pd
from IPython.display import Markdown, display

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/drive')

def printmd(string):
    display(Markdown(string))

Mounted at /content/drive


In [2]:
from sklearn.preprocessing import OneHotEncoder
## Read Data

df_issue = pd.read_csv(r'/path/to/Labeled/Data/AllSampledProjectLabeldeddata_English_Feature_Agreed.csv')

print('\n Total issues/comments:', df_issue.shape[0])
display(df_issue.sample(10))


 Total issues/comments: 2418


type                                url  number  \
366        Scientific     sagemathinc/cocalc/issues/1874    1874   
2257  None-Scientific              atom/atom/issues/8553    8553   
1689  None-Scientific     microsoft/vscode/issues/116420  116420   
103        Scientific      sagemathinc/cocalc/issues/375     375   
727        Scientific  jupyterlab/jupyterlab/issues/4730    4730   
1123       Scientific       jupyter/notebook/issues/4851    4851   
1438       Scientific        rstudio/rstudio/issues/7733    7733   
1812  None-Scientific      microsoft/vscode/issues/78214   78214   
1108       Scientific       jupyter/notebook/issues/4820    4820   
868        Scientific        rstudio/rstudio/issues/3825    3825   

                                                  title  \
366   Object doesn't support property or method 'inc...   
2257  Add API for custom "virtual" lines (like those...   
1689  Explore fine grained semantic searching of sym...   
103   course directory assignment search (should be ...   
727                     Can't disable built-in Find bar   
1123                                       Kernal Error   
1438                    Add latex autocompletion in Rmd   
1812  Add promptPath as user input type for launch c...   
1108  Use Ctrl-A to select all cells so then they ca...   
868   Adapt to stricter YAML header parsing in Pando...   

                                                   user  \
366   {'login': 'haraldschilly', 'id': 207405, 'node...   
2257  {'login': 'mangecoeur', 'id': 743508, 'node_id...   
1689  {'login': 'aszenz', 'id': 25319264, 'node_id':...   
103   {'login': 'williamstein', 'id': 1276278, 'node...   
727   {'login': 'basnijholt', 'id': 6897215, 'node_i...   
1123  {'login': 'ietuday', 'id': 14893665, 'node_id'...   
1438  {'login': 'enixam', 'id': 48123899, 'node_id':...   
1812  {'login': 'letmaik', 'id': 530988, 'node_id': ...   
1108  {'login': 'benthayer', 'id': 6099299, 'node_id...   
868   {'login': 'jmcphers', 'id': 470418, 'node_id':...   

                                                 labels  comments  \
366   [{'id': 387113698, 'node_id': 'MDU6TGFiZWwzODc...         0   
2257  [{'id': 118249520, 'node_id': 'MDU6TGFiZWwxMTg...         2   
1689  [{'id': 605277194, 'node_id': 'MDU6TGFiZWw2MDU...         3   
103   [{'id': 279813194, 'node_id': 'MDU6TGFiZWwyNzk...         1   
727   [{'id': 390791434, 'node_id': 'MDU6TGFiZWwzOTA...         7   
1123  [{'id': 2847049241, 'node_id': 'MDU6TGFiZWwyOD...         3   
1438  [{'id': 708758556, 'node_id': 'MDU6TGFiZWw3MDg...         3   
1812  [{'id': 427411234, 'node_id': 'MDU6TGFiZWw0Mjc...         5   
1108  [{'id': 2847049241, 'node_id': 'MDU6TGFiZWwyOD...         0   
868   [{'id': 890292775, 'node_id': 'MDU6TGFiZWw4OTA...         4   

                     created_at                 updated_at  \
366   2017-04-19 12:34:21+00:00  2017-04-19 18:58:38+00:00   
2257  2015-08-28 10:29:19+00:00  2018-03-30 19:37:03+00:00   
1689  2021-02-11 07:28:12+00:00  2021-05-31 23:32:48+00:00   
103   2016-01-15 20:31:33+00:00  2016-07-05 21:37:55+00:00   
727   2018-06-14 21:09:16+00:00  2019-08-07 21:48:09+00:00   
1123  2019-08-31 09:34:11+00:00  2021-03-30 00:15:28+00:00   
1438  2020-09-04 14:51:33+00:00  2021-02-19 03:18:21+00:00   
1812  2019-07-30 20:25:24+00:00  2019-11-25 09:22:34+00:00   
1108  2019-08-11 23:32:24+00:00  2021-03-29 00:16:18+00:00   
868   2018-11-05 21:10:23+00:00  2019-09-13 13:38:28+00:00   

                      closed_at  ... mention_comments                  roles  \
366   2017-04-19 18:58:38+00:00  ...                0  ['Usability', 'None']   
2257  2017-09-10 15:43:08+00:00  ...                0          ['Usability']   
1689  2021-04-16 02:35:10+00:00  ...                0          ['Usability']   
103   2016-07-05 21:37:55+00:00  ...                0  ['None', 'Usability']   
727   2019-04-27 00:04:44+00:00  ...                2          ['Usability']   
1123  2019-09-02 11:43:21+00:00  ...         

For this research question, we need a group of data so we removed the rest of features (unused columns).
Variables for this RQ:
- Number of posted issues
- Number of posted comments
- Number of years on GitHub
- Number of repositories owned
- User’s role in the project 

In [4]:

# Feild that I need for this Research Question: usability_role (IV), username, number of posted issues (number_posted_issues),
#                                               number of posted comment(number_comments), joined years (years_member), 
#                                               number of repositories (number_repositories), user's role (author_association) 



df_issue = df_issue.drop(columns=['number','comments','how_to_present','Sentiment','tones',\
                                  'comment_participants','first_comment_days','last_comment_seconds',\
                                  'last_comment_days','first_comment_seconds','first_comment_seconds',\
                                  'last_comment_days','type','url','title','user','labels','created_at','closed_at',\
                                  'mention_issue','body','Usability','NielsenHeuristics','reactions','link_issue',\
                                  'link_comments','mention_comments','all_mention','all_links',\
                                 'first_comment_date','last_comment_date','number_participants','first_comment_date'\
                                  ,'last_comment_date','Usability','roles','usability_role',\
                                  'Agree_NielsenHeuristics','File_name'])

df_issue.head()

updated_at author_association      project_name  \
0  2021-05-03 00:10:18+00:00             MEMBER  Jupyter_Notebook   
1  2021-04-14 00:15:21+00:00               NONE  Jupyter_Notebook   
2  2021-04-14 00:15:20+00:00        CONTRIBUTOR  Jupyter_Notebook   
3  2021-05-04 00:06:22+00:00               NONE  Jupyter_Notebook   
4  2021-04-17 00:16:00+00:00               NONE  Jupyter_Notebook   

                                       issue_content    username  \
0            Update git hooks-git hooks refer invoke   takluyver   
1  Improve keyboard shortcut accessibility-keyboa...   naught101   
2  writable attribute file entries available note...  ssanderson   
3  Shift Tab behavior unhandy functions-tool tip ...      vdrhtc   
4  unable adjust width custom css terminal-shrink...   cailiang9   

   number_posted_issues  years_member  number_repositories  number_comments  \
0                    35            11                  376             1804   
1                     1            12                   57                0   
2                     4             9                  104               31   
3                     1             8                   30                1   
4                    10             8                   37               15   

  Agree_Usability            Agree_roles Agree_usability_role  
0            None  ['None', 'Usability']            Both_role  
1       Usability          ['Usability']            Usability  
2            None               ['None']        Non_usability  
3       Usability          ['Usability']            Usability  
4       Usability  ['Usability', 'None']            Both_role

**Data preperation**

In [5]:
def prepare_data_for_project(project, df_issue):
    print('Preparing data for', project, '...')
    df_analyze_posts = df_issue.loc[df_issue['project_name'] == project]
    print("All: ",len(df_analyze_posts))
    df_analyze_posts = df_analyze_posts.drop_duplicates(keep='last')
    print("Unify usability author: ",len(df_analyze_posts))
    display(df_analyze_posts.sample(10))
    return df_analyze_posts

VsCode_data = prepare_data_for_project('VsCode', df_issue)

Atom_data = prepare_data_for_project('Atom', df_issue)

Jupyter_Lab_data = prepare_data_for_project('Jupyter_Lab', df_issue)

Google_Colab_data = prepare_data_for_project('Google_Colab', df_issue)

CoCalc_data = prepare_data_for_project('CoCalc', df_issue)

Scientific_data = pd.concat([Jupyter_Lab_data, Google_Colab_data,CoCalc_data])
print('number of Scientific data:', len(Scientific_data))

NoneScientific_data = pd.concat([VsCode_data, Atom_data])
print('number of None Scientific data:', len(NoneScientific_data))

All_data = pd.concat([VsCode_data, Atom_data, Jupyter_Lab_data, Google_Colab_data,CoCalc_data])
print('number of All data:', len(All_data))

# For Statistical test (Chi-squared + post hoc), we need these aggregation to use data in R. So, we save it
Scientific_data.to_csv(r"/path/to/Labeled/Data/NotebookProjects_data_RQ4.csv")
NoneScientific_data.to_csv(r"/path/to/Labeled/Data/CodeEditorProjects_data_RQ4.csv")
All_data.to_csv(r"/path/to/Labeled/Data/All_Data_RQ4.csv")


Preparing data for VsCode ...
All:  385
Unify usability author:  383


updated_at author_association project_name  \
1838  2019-10-28 10:28:20+00:00             MEMBER       VsCode   
1787  2019-12-16 21:14:31+00:00               NONE       VsCode   
1777  2021-01-26 23:41:29+00:00               NONE       VsCode   
1983  2017-11-18 04:57:54+00:00               NONE       VsCode   
1869  2018-11-26 16:13:27+00:00               NONE       VsCode   
1781  2020-01-18 10:46:43+00:00               NONE       VsCode   
1970  2017-11-18 03:44:14+00:00               NONE       VsCode   
1751  2020-09-21 23:23:15+00:00               NONE       VsCode   
1839  2019-05-02 14:15:55+00:00               NONE       VsCode   
2014  2017-11-17 23:20:07+00:00               NONE       VsCode   

                                          issue_content     username  \
1838  Allow extensions add floating button specific ...      bpasero   
1787    correctly parse tsx file React hooks-  Versi...     woohling   
1777  quickfix shorcut key-need navigation shorcut k...       windwp   
1983  endregion Unexpected preprocessor directive As...      Azlbert   
1869  Error validating file invalid regular expressi...       flcdrg   
1781  GIT recognized-Issue Type Bug Install install ...     andytael   
1970  EditorFontSize equivalent Explorer text-good s...  rafnogueira   
1751  Git submodules updating GUI-Issue Type Bug wor...      bviktor   
1839  Search field structure panel-Issue Type Bug us...      ylaizet   
2014  app logo decent-  Version width app logo relev...    PhamBaTho   

      number_posted_issues  years_member  number_repositories  \
1838                  3082            10                  113   
1787                     1             6                   15   
1777                     1             7                   41   
1983                     1             5                    6   
1869                     1            11                  239   
1781                     1             7                    5   
1970                     2             8                   19   
1751                     2             8                   21   
1839                     2             6                   11   
2014                     1             8                    4   

      number_comments Agree_Usability            Agree_roles  \
1838            25400       Usability  ['Usability', 'None']   
1787                3            None               ['None']   
1777                2       Usability          ['Usability']   
1983                0            None               ['None']   
1869                0            None               ['None']   
1781                3            None               ['None']   
1970                0       Usability          ['Usability']   
1751                8            None               ['None']   
1839                2            None               ['None']   
2014                0       Usability          ['Usability']   

     Agree_usability_role  
1838            Both_role  
1787        Non_usability  
1777            Usability  
1983        Non_usability  
1869        Non_usability  
1781        Non_usability  
1970            Usability  
1751        Non_usability  
1839        Non_usability  
2014            Usability

Preparing data for Atom ...
All:  375
Unify usability author:  375


updated_at author_association project_name  \
2265  2018-03-30 14:36:55+00:00               NONE         Atom   
2211  2018-04-09 20:14:36+00:00               NONE         Atom   
2147  2018-04-01 19:36:33+00:00               NONE         Atom   
2196  2018-04-06 02:36:10+00:00               NONE         Atom   
2317  2019-01-20 21:51:44+00:00        CONTRIBUTOR         Atom   
2052  2020-07-03 17:24:31+00:00               NONE         Atom   
2303  2019-01-22 15:47:06+00:00               NONE         Atom   
2202  2018-04-08 00:36:09+00:00               NONE         Atom   
2288  2019-01-21 02:52:10+00:00               NONE         Atom   
2398  2019-01-27 03:57:11+00:00        CONTRIBUTOR         Atom   

                                          issue_content     username  \
2265  Linux Mint freezes-Somewhat randomly annoyingl...     aldelval   
2211  Metrics disabled default update-appears commun...    beautispy   
2147  Uncaught Error read property object undefinedT...        Zsfzy   
2196  Uncaught Error Invalid text decoration iterato...   petersowah   
2317    wo start Fedora-think messed installed local...        sryze   
2052  facing issue installing bit bit windows-Progra...  kaushik1810   
2303  updating following deprecation notification co...       rukshn   
2202    terminal command works app running-  reprodu...         s2t2   
2288  Packages Install options Settings lock-updates...      karai17   
2398  cmd windows work windows screen-cmd windows wo...    jbarnette   

      number_posted_issues  years_member  number_repositories  \
2265                     1             8                    0   
2211                     1             5                    0   
2147                     1             5                   20   
2196                     1             7                  190   
2317                     8             8                   43   
2052                     1             1                    1   
2303                     2             9                   39   
2202                     1             9                  706   
2288                    14            10                   43   
2398                    40            13                   75   

      number_comments Agree_Usability            Agree_roles  \
2265                2            None               ['None']   
2211                0            None               ['None']   
2147                0            None               ['None']   
2196                0            None               ['None']   
2317               16            None               ['None']   
2052                0            None               ['None']   
2303                2            None               ['None']   
2202                5            None               ['None']   
2288               29            None               ['None']   
2398               95            None  ['None', 'Usability']   

     Agree_usability_role  
2265        Non_usability  
2211        Non_usability  
2147        Non_usability  
2196        Non_usability  
2317        Non_usability  
2052        Non_usability  
2303        Non_usability  
2202        Non_usability  
2288        Non_usability  
2398            Both_role

Preparing data for Jupyter_Lab ...
All:  351
Unify usability author:  351


...

Preparing data for Google_Colab ...
All:  294
Unify usability author:  294


updated_at author_association  project_name  \
694   2018-06-12 15:28:19+00:00               NONE  Google_Colab   
1205  2019-12-08 04:17:16+00:00               NONE  Google_Colab   
1360  2020-06-15 18:26:17+00:00               NONE  Google_Colab   
876   2020-09-28 23:06:10+00:00               NONE  Google_Colab   
1627  2021-05-23 14:02:20+00:00               NONE  Google_Colab   
1332  2020-11-20 00:01:51+00:00               NONE  Google_Colab   
1379  2020-07-09 19:02:35+00:00               NONE  Google_Colab   
1639  2021-06-10 21:03:11+00:00               NONE  Google_Colab   
1416  2020-08-09 20:11:49+00:00               NONE  Google_Colab   
863   2021-02-11 09:43:20+00:00               NONE  Google_Colab   

                                          issue_content          username  \
694   Validation programming exercise shows error-ru...        koshygeoji   
1205  work-Bug report   questions   usage use stacko...  cytotechnologist   
1360  Feature Request Tracking GPU memory usage-  bu...       wishforgood   
876     files   Drive-example open file   drive txt ...            phly95   
1627     pro free-Vulnerability allows use    termin...          MrZethaz   
1332  Issue reading large file-   read hf file GB er...             vsanz   
1379  Consider line wrapping errors message default ...            shoyer   
1639  network didnot learning traning-yolov training...          totoadel   
1416  selection paste error-rows selected rows selec...          vainaijr   
863   set higher shared memory avoid RuntimeError Da...         discdiver   

      number_posted_issues  years_member  number_repositories  \
694                      1             3                   17   
1205                     1             2                    1   
1360                     1             7                   23   
876                      1             8                    3   
1627                     1             0                    3   
1332                     1             8                    7   
1379                     2            10                   82   
1639                     1             5                   10   
1416                    21             3                   19   
863                      3             7                  128   

      number_comments Agree_Usability            Agree_roles  \
694                 0            None               ['None']   
1205                0            None               ['None']   
1360                1       Usability          ['Usability']   
876                 0            None               ['None']   
1627                1            None               ['None']   
1332                0            None               ['None']   
1379                2       Usability          ['Usability']   
1639                0            None               ['None']   
1416               13            None  ['Usability', 'None']   
863                 4            None               ['None']   

     Agree_usability_role  
694         Non_usability  
1205        Non_usability  
1360            Usability  
876         Non_usability  
1627        Non_usability  
1332        Non_usability  
1379            Usability  
1639        Non_usability  
1416            Both_role  
863         Non_usability

Preparing data for CoCalc ...
All:  331
Unify usability author:  331


updated_at author_association project_name  \
420   2017-06-26 19:25:00+00:00        CONTRIBUTOR       CoCalc   
1287  2020-04-10 04:58:07+00:00        CONTRIBUTOR       CoCalc   
1250  2020-02-26 21:42:22+00:00        CONTRIBUTOR       CoCalc   
340   2017-03-08 23:27:40+00:00        CONTRIBUTOR       CoCalc   
191   2016-09-22 03:26:32+00:00               NONE       CoCalc   
330   2017-03-07 01:54:35+00:00        CONTRIBUTOR       CoCalc   
295   2017-01-06 13:40:25+00:00        CONTRIBUTOR       CoCalc   
225   2016-12-19 05:16:50+00:00        CONTRIBUTOR       CoCalc   
1153  2019-10-03 16:49:08+00:00        CONTRIBUTOR       CoCalc   
1183  2020-07-21 23:25:19+00:00        CONTRIBUTOR       CoCalc   

                                          issue_content             username  \
420   rename blog-Got blog com blog Notice title fav...         williamstein   
1287  nbgrader provide option run tests instructor p...         williamstein   
1250  hub api client → project readfile → read prope...        haraldschilly   
340   change defaultmode modename sage worksheet sni...         williamstein   
191   URLs working redirecting-Edited John following...          GregoryBard   
330   topsort Store cycle computed values-exception ...        haraldschilly   
295   add java syntax highlighting-add java syntax h...         williamstein   
225   major revamp homepage-Header Login general blu...  timothyclemansinsea   
1153  terminal x tell matplotlib draw-matplotlib dif...        haraldschilly   
1183  chat drag drop image upload moves buttons-drag...        haraldschilly   

      number_posted_issues  years_member  number_repositories  \
420                    819            10                  116   
1287                   819            10                  116   
1250                   567            11                  100   
340                    819            10                  116   
191                      7             5                    1   
330                    567            11                  100   
295                    819            10                  116   
225                     84             6                   21   
1153                   567            11                  100   
1183                   567            11                  100   

      number_comments Agree_Usability            Agree_roles  \
420              2381            None  ['None', 'Usability']   
1287             2381            None  ['None', 'Usability']   
1250              966            None  ['Usability', 'None']   
340              2381            None  ['None', 'Usability']   
191                29            None               ['None']   
330               966            None  ['Usability', 'None']   
295              2381            None  ['None', 'Usability']   
225                77            None  ['Usability', 'None']   
1153              966            None  ['Usability', 'None']   
1183              966       Usability  ['Usability', 'None']   

     Agree_usability_role  
420             Both_role  
1287            Both_role  
1250            Both_role  
340             Both_role  
191         Non_usability  
330             Both_role  
295             Both_role  
225             Both_role  
1153            Both_role  
1183            Both_role

number of Scientific data: 976
number of None Scientific data: 758
number of All data: 1734


**Data Analysis**


In [7]:
pip install scikit-posthocs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:

import statsmodels.api as sa
import statsmodels.formula.api as sfa
import scikit_posthocs as sp
import scipy.stats as ss
from scipy import stats

# column name group_name column in df how many groups inthe column.
def kruskal_with_post_hoc(df_analyze_posts, group_name, value_name, alpha=0.05):
    
    print("This is Kruskal test with Post hoc")
    
    for name, group in df_analyze_posts.groupby(group_name):
        groups = [group[value_name].values for \
                           name, group in df_analyze_posts.groupby(group_name)] 
        num_groups = len(groups)
        if num_groups >= 2:
            
            stat, p = ss.kruskal(*groups)
            eta2 = (stat - num_groups + 1)/(len(df_analyze_posts) - num_groups)
              
            if p <= alpha:
                printmd ('H = ' + str(round(stat, 3)) + 
                         '  <span style="color:red">p = ' + str(round(p, 10))+ '</span>' +
                         '  eta2 = ' + str(round(eta2, 3)))
                
                posthoc = \
                    sp.posthoc_mannwhitney(df_analyze_posts, val_col=value_name, 
                                           group_col=group_name, p_adjust = 'holm')
                display(posthoc)
                display(df_analyze_posts.groupby(group_name).mean()[value_name])
            else:
                printmd ('H = ' + str(round(stat, 3)) + 
                         '  p = ' + str(round(p, 5)) +
                         '  eta2 = ' + str(round(eta2, 3)))
                        
            return p
        else: 
            p = 0
            return p 
        
####################################################
def anova_with_post_hoc(df_analyze_posts, group_name, value_name, formula, alpha=0.05):
    
    #formula = "\'"+formula+"\'"
    print("This is ANOVA test with Post hoc")
    for name, group in df_analyze_posts.groupby(group_name):
        groups = [group[value_name].values for \
                           name, group in df_analyze_posts.groupby(group_name)] 
        num_groups = len(groups)
        if num_groups >= 2:
            lm = sfa.ols(formula, data=df_analyze_posts).fit()
            anova = sa.stats.anova_lm(lm)
            p_value = anova['PR(>F)'][group_name]
            if p_value <= alpha:
                printmd ('  <span style="color:red">p-value = ' + str(round(p_value, 3))+ '</span>')
            else:
                printmd ('p-value = ' + str(round(p_value, 3)))
            
            print(anova)  
            p_value = round(p_value, 2)
            if p_value <= alpha :
                ## The results tell us that there is a significant difference between groups means (p = 4.49e-17), 
                ## but does not tell us the exact group pairs which are different in means. 
                ## To obtain pairwise group differences, we will carry out a posteriori (post hoc) analysis using 
                ## scikits-posthocs package.
                printmd ('Mean_sq = ' + str(round(anova['F'][group_name], 3)) + 
                         '  <span style="color:red"> , p_value = ' + str(round(p_value, 10))+ '</span>' )
                data = sp.posthoc_ttest(df_analyze_posts, val_col = value_name, group_col = group_name, p_adjust='bonferroni')
                display(data)
                print(df_analyze_posts[[group_name,value_name]].groupby(group_name).mean()) 
            else: 
                 printmd ('Mean_sq = ' + str(round(anova['F'][group_name], 3)) + 
                 ' , p_value = ' + str(round(p_value, 5)))
                         
            return p_value
        else: 
            p = 0
            return p_value         

####################################################        
def Shapiro_Wilk_test(df_analyze_posts, group_name, alpha=0.05):
    ## The Shapiro-Wilk test is a test of normality. It is used to determine whether or not a sample comes from a normal
    ## distribution. This function returns a test statistic and a corresponding p-value. If the p-value is below a 
    ## certain significance level, then we have sufficient evidence to say that the sample data does not come from a 
    ## normal distribution. From the output we can see that the test statistic is 0.9927 and the corresponding p-value
    ## is 0.8689. Since the p-value is not less than .05, we fail to reject the null hypothesis. 
    
    y = []

    df_analyze_posts[group_name] = df_analyze_posts[group_name].fillna(0.0)
    for index, row in df_analyze_posts.iterrows():
        y.append(np.float64(row[group_name]))

    shapiro_test = stats.shapiro(y)
    
    if shapiro_test.pvalue <= alpha:
        printmd ('Test Statistic = ' + str(round(shapiro_test.statistic, 3)) + 
                 '  <span style="color:red">p = ' + str(round(shapiro_test.pvalue, 10))+ '</span>')
    else:
        printmd ('Test Statistic = ' + str(round(shapiro_test.statistic, 3)) + 
                 '  p = ' + str(round(shapiro_test.pvalue, 5)))
        
    return shapiro_test.pvalue
  

In [9]:
print("##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################")
print("### dependent variable: number of posted issues (number_posted_issues) , Independent variable: Authors Usability role => Kruskal-Wallis or ANOVA + posthoc ###")
alpha = 0.05
## First test for normal distribution of dependent variable using Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc**
printmd('<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(All_data,"number_posted_issues")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(All_data, 'Agree_usability_role','number_posted_issues','number_posted_issues ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(All_data, 'Agree_usability_role','number_posted_issues')

#################################################    
printmd('<p><span style="color:green">#######################    Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(Scientific_data,"number_posted_issues")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(Scientific_data, 'Agree_usability_role','number_posted_issues','number_posted_issues ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(Scientific_data, 'Agree_usability_role','number_posted_issues')
    
#################################################
printmd('<p><span style="color:green">#######################    None Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(NoneScientific_data,"number_posted_issues")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(NoneScientific_data, 'Agree_usability_role','number_posted_issues','number_posted_issues ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(NoneScientific_data, 'Agree_usability_role','number_posted_issues')
    

##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################
### dependent variable: number of posted issues (number_posted_issues) , Independent variable: Authors Usability role => Kruskal-Wallis or ANOVA + posthoc ###


<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>

Test Statistic = 0.501  <span style="color:red">p = 0.0</span>

This is ANOVA test with Post hoc


  <span style="color:red">p-value = 0.0</span>

...

  <span style="color:red">p-value = 0.0</span>

                         df        sum_sq       mean_sq           F  \
Agree_usability_role    2.0  1.084334e+08  5.421668e+07  327.152561   
Residual              755.0  1.251208e+08  1.657229e+05         NaN   

                             PR(>F)  
Agree_usability_role  4.737784e-103  
Residual                        NaN  


Mean_sq = 327.153  <span style="color:red"> , p_value = 0.0</span>

Non_usability     Both_role     Usability
Non_usability   1.000000e+00  2.162328e-89  2.493359e-01
Both_role       2.162328e-89  1.000000e+00  1.212680e-22
Usability       2.493359e-01  1.212680e-22  1.000000e+00

                      number_posted_issues
Agree_usability_role                      
Both_role                      1322.893333
Non_usability                    63.114334
Usability                        20.556701


In [10]:
print("##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################")
print("### dependent variable: number of posted comments (number_comments) , Independent variable: Authors Usability role => Kruskal-Wallis or ANOVA + posthoc ###")
alpha = 0.05
## First test for normal distribution of dependent variable using Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc**
printmd('<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(All_data,"number_comments")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(All_data, 'Agree_usability_role','number_comments','number_comments ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(All_data, 'Agree_usability_role','number_comments')

#################################################    
printmd('<p><span style="color:green">#######################    Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(Scientific_data,"number_comments")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(Scientific_data, 'Agree_usability_role','number_comments','number_comments ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(Scientific_data, 'Agree_usability_role','number_comments')
    
#################################################
printmd('<p><span style="color:green">#######################    None Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(NoneScientific_data,"number_comments")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(NoneScientific_data, 'Agree_usability_role','number_comments','number_comments ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(NoneScientific_data, 'Agree_usability_role','number_comments')
    

##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################
### dependent variable: number of posted comments (number_comments) , Independent variable: Authors Usability role => Kruskal-Wallis or ANOVA + posthoc ###


<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>

Test Statistic = 0.325  <span style="color:red">p = 0.0</span>

This is ANOVA test with Post hoc


  <span style="color:red">p-value = 0.0</span>

...

  <span style="color:red">p-value = 0.0</span>

                         df        sum_sq       mean_sq          F  \
Agree_usability_role    2.0  7.486815e+09  3.743408e+09  328.12362   
Residual              755.0  8.613439e+09  1.140853e+07        NaN   

                             PR(>F)  
Agree_usability_role  2.817098e-103  
Residual                        NaN  


Mean_sq = 328.124  <span style="color:red"> , p_value = 0.0</span>

Non_usability     Both_role     Usability
Non_usability   1.000000e+00  1.924927e-89  2.613285e-01
Both_role       1.924927e-89  1.000000e+00  2.648957e-22
Usability       2.613285e-01  2.648957e-22  1.000000e+00

                      number_comments
Agree_usability_role                 
Both_role                10886.693333
Non_usability              415.629693
Usability                   79.092784


In [11]:
print("##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################")
print("### dependent variable: joind year  (years_member) , Independent variable: Authors Usability role => Kruskal-Wallis or ANOVA + posthoc ###")
alpha = 0.05
## First test for normal distribution of dependent variable using Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc**
printmd('<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(All_data,"years_member")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(All_data, 'Agree_usability_role','years_member','years_member ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(All_data, 'Agree_usability_role','years_member')

#################################################    
printmd('<p><span style="color:green">#######################    Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(Scientific_data,"years_member")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(Scientific_data, 'Agree_usability_role','years_member','years_member ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(Scientific_data, 'Agree_usability_role','years_member')
    
#################################################
printmd('<p><span style="color:green">#######################    None Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(NoneScientific_data,"years_member")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(NoneScientific_data, 'Agree_usability_role','years_member','years_member ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(NoneScientific_data, 'Agree_usability_role','years_member')
    

##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################
### dependent variable: joind year  (years_member) , Independent variable: Authors Usability role => Kruskal-Wallis or ANOVA + posthoc ###


<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>

Test Statistic = 0.95  <span style="color:red">p = 0.0</span>

This is ANOVA test with Post hoc


  <span style="color:red">p-value = 0.0</span>

...

  <span style="color:red">p-value = 0.0</span>

                         df       sum_sq     mean_sq          F        PR(>F)
Agree_usability_role    2.0   281.564591  140.782296  15.142917  3.564459e-07
Residual              755.0  7019.164960    9.296907        NaN           NaN


Mean_sq = 15.143  <span style="color:red"> , p_value = 0.0</span>

Non_usability     Both_role  Usability
Non_usability   1.000000e+00  2.705598e-07   0.437664
Both_role       2.705598e-07  1.000000e+00   0.000381
Usability       4.376643e-01  3.807001e-04   1.000000

                      years_member
Agree_usability_role              
Both_role                 9.533333
Non_usability             7.498294
Usability                 8.000000


In [12]:
print("##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################")
print("### dependent variable: number of repositories  (number_repositories) , Independent variable: Agree_usability_role => Kruskal-Wallis or ANOVA + posthoc ###")
alpha = 0.05
## First test for normal distribution of dependent variable using Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc**
printmd('<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(All_data,"number_repositories")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(All_data, 'Agree_usability_role','number_repositories','number_repositories ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(All_data, 'Agree_usability_role','number_repositories')

#################################################    
printmd('<p><span style="color:green">#######################    Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(Scientific_data,"number_repositories")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(Scientific_data, 'Agree_usability_role','number_repositories','number_repositories ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Agree_usability_role', 'time_to_first_comment')
    kruskal_with_post_hoc(Scientific_data, 'Agree_usability_role','number_repositories')
    
#################################################
printmd('<p><span style="color:green">#######################    None Scientific Project    ########################</span></p>')

p_value = Shapiro_Wilk_test(NoneScientific_data,"number_repositories")
# p-value is less than .05 the data are normal distribution
if p_value <= alpha: 
    # Normal distribution : anova_with_post_hoc(df_issue, 'Agree_usability_role','number_posted_issues', 'number_posted_issues ~ Agree_usability_role')
    anova_with_post_hoc(NoneScientific_data, 'Agree_usability_role','number_repositories','number_repositories ~ Agree_usability_role')
else: 
    # Not Normal distribution: kruskal_with_post_hoc(df_analyze_posts, 'Sentiment', 'time_to_first_comment')
    kruskal_with_post_hoc(NoneScientific_data, 'Agree_usability_role','number_repositories')
    

##################  Shapiro-Wilk test then use Kruskal-Wallis or ANOVA + posthoc #########################
### dependent variable: number of repositories  (number_repositories) , Independent variable: Agree_usability_role => Kruskal-Wallis or ANOVA + posthoc ###


<p><span style="color:green">#######################    All Sampled Project    ########################</span></p>

Test Statistic = 0.632  <span style="color:red">p = 0.0</span>

This is ANOVA test with Post hoc


  <span style="color:red">p-value = 0.0</span>

...

  <span style="color:red">p-value = 0.0</span>

                         df        sum_sq       mean_sq          F    PR(>F)
Agree_usability_role    2.0  1.564577e+05  78228.828283  14.010143  0.000001
Residual              755.0  4.215715e+06   5583.728048        NaN       NaN


Mean_sq = 14.01  <span style="color:red"> , p_value = 0.0</span>

Non_usability     Both_role  Usability
Non_usability   1.000000e+00  4.703252e-07   1.000000
Both_role       4.703252e-07  1.000000e+00   0.000099
Usability       1.000000e+00  9.871058e-05   1.000000

                      number_repositories
Agree_usability_role                     
Both_role                       98.773333
Non_usability                   50.501706
Usability                       51.731959
